In [ ]:
import os
import sys
import random
import numpy as np
from glob import glob

seed = 2882
random.seed(seed)
np.random.seed(seed)

In [ ]:
dataset_path = './data/preprocessed'
image_dirs = os.listdir(dataset_path)
img_path_list = []
labe_list = []
for image_dir in image_dirs:
    cur_dir_path = os.path.join(dataset_path,image_dir)
    cur_img_list = glob(os.path.join(cur_dir_path,'*.jpg'))
    img_path_list += cur_img_list
    labe_list += [image_dir.split('_')]*len(cur_img_list)
    print(image_dir, len(cur_img_list), labe_list[-1])
print('Total imgages:', len(img_path_list))

### Mutil Label

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(FIX_ME)
print(y)
print(mlb.classes_)

### Load Images

In [ ]:
import numpy as np
from keras.preprocessing.image import img_to_array
from PIL import Image as pil_image

img_w = 128
img_h = 128
x = np.empty((len(img_path_list), img_w, img_h, 3), dtype='float32')
for ind, p in enumerate(img_path_list):
    x[ind] = img_to_array(pil_image.open(p))/255.
print(x.shape)

### Shuffle

In [ ]:
def shuffle(x,y):
    combined = list(zip(x,y))
    random.shuffle(combined)
    
    x[:], y[:] = zip(*combined)
    return x, y
x, y = shuffle(x,y)

In [ ]:
y

In [ ]:
split_ration = 0.2
split_ind = int(len(y)*(1-split_ration))
x_train = x[:split_ind]
y_tain = y[:split_ind]
x_val = x[split_ind:]
y_val = y[split_ind:]

### Build Model

In [ ]:
def custom_vgg_model(input_shape, output_shape):
    from keras.applications.vgg16 import VGG16
    from keras import Model
    from keras.layers import Dense, Flatten

    vgg16_base = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    hid = vgg16_base.output
    hid = Flatten(name='flatten')(hid)
    hid = Dense(512, activation='relu', name='fc1')(hid)
    hid = Dense(512, activation='relu', name='fc2')(hid)
    out = Dense(output_shape, activation=FIX_ME, name='predictions')(hid)

    model = Model(inputs=vgg16_base.input, outputs=out)
    model.compile(optimizer='adadelta', loss=FIX_ME, metrics=['accuracy'])

    return model

model = custom_vgg_model(input_shape=x[0].shape, output_shape=len(y[0]))
model.summary()

In [ ]:
def test(model):
    from matplotlib import pyplot as plt
    %matplotlib inline
    plt.rcParams['figure.figsize'] = (18,18)

    f, ax = plt.subplots(1,3, squeeze=False)

    test_img_path_list = glob('./data/testing/*.jpg')
    print(mlb.classes_)
    for ind, img_path in enumerate(test_img_path_list):
        cur_img = pil_image.open(img_path).resize((img_w, img_h))
        ax[0][ind].imshow(cur_img)
        res = model.predict(np.asarray([img_to_array(cur_img)/255.]))
        file_name = os.path.basename(img_path)
        print(file_name.split('.')[0], list(map("{0:.4f}".format,res[0])), [ mlb.classes_[ind] for ind in np.where(res[0]>0.5)[0]])

In [ ]:
import keras 
batch_size = 32
epochs = 10
best_model_path = './best_model.h5'
checkpoint = keras.callbacks.ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, verbose=1)
model = custom_vgg_model(input_shape=x[0].shape, output_shape=len(y[0]))

history = model.fit(x=x_train,
                    y=y_tain,
                    validation_data=(x_val,y_val),
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[checkpoint])

In [ ]:
from keras.models import load_model
cur_model = load_model('./best_model.h5')
test(cur_model)

### With class weight

In [ ]:
from collections import Counter
from functools import reduce
y_flatten = reduce(lambda x,y: x+y,labe_list)
Counter(y_flatten)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weight = compute_class_weight(FIX_ME)
class_weight

In [ ]:
import keras 
batch_size = 32
epochs = 10
best_model_path = './best_model_class_weight.h5'
checkpoint = keras.callbacks.ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, verbose=1)
model = custom_vgg_model(input_shape=x[0].shape, output_shape=len(y[0]))

history = model.fit(x=x_train,
                    y=y_tain,
                    validation_data=(x_val,y_val),
                    batch_size=batch_size,
                    epochs=epochs,
                    class_weight=class_weight,
                    callbacks=[checkpoint])

In [ ]:
from keras.models import load_model
cur_model = load_model('./best_model_class_weight.h5')
test(cur_model)